In [ ]:
!pip install torch torchvision
!pip install tqdm
!pip install tensorboard

In [ ]:
import os.path
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torchvision.datasets import ImageFolder
from tqdm import tqdm
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
import numpy as np
from torchvision.models import resnet18, ResNet18_Weights, vgg11 , VGG11_Weights, resnet50, ResNet50_Weights,  mobilenet_v2, MobileNet_V2_Weights
from torchvision.transforms import (Compose,
                                    RandomResizedCrop,
                                    RandomHorizontalFlip,
                                    ToTensor,
                                    Normalize,
                                    Resize,
                                    CenterCrop
                                    )
from sklearn.metrics import roc_curve, auc
import argparse
import shutil
import warnings
warnings.filterwarnings("ignore")

# Hyper parameter

Các hyper parameter mặc định cho việc train. Trong đó có một vài cái là đường dẫn file kaggle

In [ ]:
def parse_arg():
    parse = argparse.ArgumentParser(description='Transfer learning')
    parse.add_argument('--epochs', '-e', type=int, default=100, help='Số lượng epoch để chạy')
    parse.add_argument('--checkpoint-dir', '-d', type=str, default='/kaggle/working/checkpoint', help='Nơi lưu checkpoint - weght của model')
    parse.add_argument('--tensorboard', '-t', type=str, default='/kaggle/working/dashboad', help='Nơi lưu file .event trực quan hóa')
    parse.add_argument('--lr', '-l', type=float, default=1e-3)
    parse.add_argument('--EarlyStopping','-s',type=int, default=70, help='Early stopping càng lớn thì train được càng nhiều')
    parse.add_argument('--batch-size','-b',type=int, default=16, help='Kích thước mỗi batch cho vào mô hình')
    args, unknown = parse.parse_known_args()
    return args


# Create DataLoader

Tiền xử lý dữ liệu bằng data loader

In [ ]:
# Đường dẫn tới dataset trên kaggle (tùy chỉnh trước khi chạy)
temp_val = '/kaggle/working/train'
temp_train = '/kaggle/working/val'

def preprocessing():
    transforms = Compose([
            RandomResizedCrop(224),
            RandomHorizontalFlip(),
            ToTensor(),
            Normalize([0.574, 0.574, 0.574], [0.169, 0.169, 0.169])
        ]), 

    dataset = ImageFolder(root=temp_train, transform=transforms)
    data_loader = DataLoader(
        dataset=dataset,
        batch_size=16,
        shuffle=True,
        num_workers=4,
        drop_last=True
    )

    return data_loader


# Function for evaluation

Đánh giá mô hình bằng metrics accuracy khi cần thiết

In [ ]:
def metrics_evaluations(writer, optimizer, epoch, score, metrics, name, model,
                        model_checkpoint_path):
    writer.add_scalar(tag=f'{name}/val', scalar_value=score, global_step=epoch)

    # show kết quả

    checkpoint = {
        'state': model.state_dict(),
        'epoch': epoch,
        'optimizer': optimizer.state_dict(),
        'best_epoch': metrics['best_epoch'],
        f'best_{name}': metrics[f'best_{name}']
    }    
    
    if metrics[f'best_{name}'] < score:
        metrics[f'best_{name}'] = score
        metrics[f'best_epoch'] = epoch
        torch.save(checkpoint, model_checkpoint_path + '/' + f'best_{name}.pt')



# Transfer learning

**Tái huấn luyện mô hình**

In [ ]:

def transfer_learning(model, model_name, classes, criterion, optimizer, parse, data_loader):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    metrics = {
                'best_accuracy' : -999,
                'best_epoch' : 0
    }
    
    for epoch in range(parse.epochs):
        print('-' * 10)
        print(f'Epoch {epoch + 1}/{parse.epochs}')

        # training
        model.train()
        loss_recorded = []
        all_pred = []
        all_labels = []

        progress_bar = tqdm(train_loader, colour='yellow')
        for iter, (images, labels) in enumerate(progress_bar):
            images = images.to(device)
            labels = labels.to(device)

            # foward
            pred = model(images)
            max_pred = torch.argmax(pred, dim=1)
            loss = criterion(pred, labels)

            # backward
            optimizer.zero_grad()
            loss.backward() 
            optimizer.step() 

            # ghi nhận loss
            all_labels.extend(labels.tolist())
            all_pred.extend(max_pred.tolist())
            loss_recorded.append(loss.item())

            progress_bar.set_description(f'Train/Batch {iter}, Loss: {loss.item():.4f}')
            writer.add_scalar(tag='Train/Loss', scalar_value=np.mean(loss_recorded),
                              global_step=epoch * len(train_loader) + iter)

        # numerical metrics
        acc = accuracy_score(all_labels, all_pred)
        writer.add_scalar(tag='Train/Accuracy', scalar_value=acc,
                          global_step=epoch * len(train_loader) + iter)
        print(f'Acc: {acc:.4f}, avg_oss: {np.mean(loss_recorded):.4f}')
        if abs(metrics['best_epoch'] - epoch) >= parse.EarlyStopping:
            break

In [ ]:
data = preprocessing()
parse = parse_arg()
classes = ['bào ngư xám + trắng', 'linh chi trắng','nấm mỡ','Đùi gà Baby (cắt ngắn)']

# criterion
criterion = nn.CrossEntropyLoss()

# Một vài chiến lược cải thiện mô hình bằng hyper parameters

- Early stopping
  + Càng nhỏ thì học càng nhanh, nhưng độ chính xác không cao
  + Càng lớn thì độ chính xác cao nhưng mô hình học càng chậm,

- learning rate
  + Càng nhỏ thì càng tối ưu nhưng chạy chậm
  + Càng lớn thì ngược lại

- Batch size
  + Càng nhỏ: Tiêu tốn ít RAM hơn, tổng quát hóa tốt hơn, nhưng chậm
  + Càng lớn: Tiêu tốn nhiều RAM hơn, dễ bị overfit
  
- optimizer
  + Adam
  + SGD
  + Adagrad
  + momemtum
    
- Đổi mô hình:
    + Mô hình càng lớn, dữ liệu càng nhỏ thì dễ bị overfiting

      Giải pháp: chọn mô hình nhỏ hơn để phù hợp với dữ liệu

# Chạy thử với mô hình mobilenet_v3_small

In [ ]:
# --------------------------------- mobilenet_v3_small ---------------------------------
from torchvision.models import mobilenet_v3_small, MobileNet_V3_Small_Weights
mobilenet = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights)
mobilenet.classifier[3] = nn.Linear(in_features=1024, out_features=len(classes), bias=True)

optimizer = optim.SGD(mobilenet.parameters(),momentum=0.9, lr=parse.lr, weight_decay=5e-4)

transfer_learning(mobilenet,
                    'mobilenet_v3_small',
                    classes,
                    criterion,
                    optimizer,
                    parse,
                    train_loader,
                    val_loader
                    )

In [ ]:

def predict_test():
    pass
    # pass
    # pass